In [1]:
import json
from tqdm import tqdm
import numpy as np

import os
import string

from pytorch_transformers.tokenization_bert import BertTokenizer

In [2]:
file_lines = open('sample_elq_dev.jsonl').readlines()
sample_elq = {}
for line in file_lines:
    line_json = json.loads(line)
    sample_elq[line_json['id']] = line_json

In [11]:
sample_elq.keys()

dict_keys(['947testa CRICKET', '948testa CRICKET', '949testa CRICKET', '950testa SOCCER', '951testa BASKETBALL', '952testa SOCCER', '953testa SOCCER', '954testa BOXING', '955testa SQUASH', '956testa SOCCER', '957testa BASEBALL', '958testa TENNIS', '959testa BASEBALL', '960testa BASEBALL', '961testa TENNIS', '962testa NFL', '963testa GOLF', '964testa GOLF', '965testa SOCCER', '966testa ATHLETICS', '967testa ATHLETICS', '968testa SOCCER', '969testa ATHLETICS', '970testa SOCCER', '971testa SOCCER', '972testa CRICKET', '973testa CRICKET', '974testa CRICKET', '975testa CRICKET', '976testa ROMANIA', '977testa POLISH', '978testa Canada', '979testa Jones', '980testa NYMEX', '981testa U.S.', '982testa Douglas', '983testa UK', '984testa Iraqi', '985testa Late', '986testa Boxing-Bruno', '987testa Soccer', '988testa Nigerian', '989testa East', '990testa Chad', '991testa Yeltsin', '992testa Lebed', '993testa Ruling', '994testa Belgrade', '995testa Top', '996testa Tajik', '997testa Polish', '998test

In [12]:
elq_sample = sample_elq['947testa CRICKET']
elq_sample

{'id': '947testa CRICKET',
 'text': "CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY . LONDON 1996-08-30 West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship . Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire . After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 . Trailing by 213 , Somerset got a solid start to their second innings before Simmons stepped in to bundle them out for 174 . Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip on their match against Yorkshi

In [3]:
fpath = 'AIDA-YAGO2-en_desc'

fname = os.path.join(fpath, 'train.json')

with open(fname) as fin:
    train = json.load(fin)

In [7]:
example = train[0]
example.keys()

dict_keys(['doc_id', 'text', 'mentions', 'start_idxs', 'end_idxs', 'wiki_ids', 'wiki_titles', 'wiki_contexts', 'start_position', 'end_position', 'span_position'])

In [22]:
example['span_position']

[[2, 2],
 [6, 6],
 [11, 11],
 [14, 15],
 [22, 22],
 [28, 28],
 [43, 43],
 [48, 49],
 [66, 66],
 [94, 94],
 [129, 130],
 [142, 143],
 [179, 179],
 [181, 181],
 [193, 193],
 [237, 237],
 [240, 242],
 [266, 266],
 [268, 268],
 [314, 314],
 [330, 330],
 [350, 350],
 [357, 357],
 [363, 363],
 [382, 382],
 [404, 404],
 [420, 420],
 [438, 438],
 [443, 443],
 [459, 459]]

In [14]:
full_example = example['text']

In [23]:
example['text'][:3]

['EU', 'rejects', 'German']

In [24]:
sent = 'EU rejects German'
sent

'EU rejects German'

In [28]:
sent[11:18]

'German'

## Tokenized elq format

- 'id': 'doc_id'
- 'text': 'text'.join
- 'mentions': index span (char) of mentions, list of lists (end idx + 1)
- 'tokenized_text_ids': tokenized text id
- 'tokenized_mention_idxs': similar to span_position, list of lists (end idx + 1)
- 'label_id': 'wiki_ids' (ELQ uses new id)
- 'wikidata_id': not include, plan to remove in code
- 'entity': 'wiki_titles'
- 'label': 'wiki_contexts'

In [3]:
tokenizer = BertTokenizer.from_pretrained(
    "bert-base-uncased", do_lower_case=True,
)

In [4]:
def get_tokenized_bounds(mentions, full_example, tokenizer):
    example_ranges = mentions
    
    char_in_mention_idx_map = [[] for _ in range(len(full_example))]
    all_mention_bounds = []
    for m, ment in enumerate(example_ranges):
        for c in range(ment[0], ment[1]):
            char_in_mention_idx_map[c].append(m)
        all_mention_bounds.append(ment[0])
        all_mention_bounds.append(ment[1])
    all_mention_bounds = [0] + all_mention_bounds + [len(full_example)]
    all_mention_bounds = list(set(all_mention_bounds))
    all_mention_bounds.sort()
    
    example_chunks = [full_example[all_mention_bounds[b]:(all_mention_bounds[b+1])] for b in range(len(all_mention_bounds) - 1)]
    chunk_idx_to_mention_idx_map = []
    bound_idx = 0
    for c, chunk in enumerate(example_chunks):
        assert bound_idx == all_mention_bounds[c]
        try:
            chunk_idx_to_mention_idx_map.append(char_in_mention_idx_map[all_mention_bounds[c]])
        except:
            print("error checkpoint")
            import pdb
            pdb.set_trace()
        bound_idx += len(chunk)
    mention_idx_to_chunk_idx_map = {}
    chunk_idx_to_tokenized_bounds = {}
    mention_idxs = []
    all_token_ids = []
    cum_len = 0
    for c, chunk in enumerate(example_chunks):
        chunk_tokens = tokenizer.encode(chunk)
        all_token_ids += chunk_tokens
        chunk_bounds = [cum_len, cum_len+len(chunk_tokens)]
        for m in chunk_idx_to_mention_idx_map[c]:
            if m not in mention_idx_to_chunk_idx_map:
                mention_idx_to_chunk_idx_map[m] = chunk_bounds
            else:
                existing_chunk_bounds = mention_idx_to_chunk_idx_map[m]
                mention_idx_to_chunk_idx_map[m] = [
                    min(existing_chunk_bounds[0], chunk_bounds[0]),
                    max(existing_chunk_bounds[1], chunk_bounds[1]),
                ]
        cum_len += len(chunk_tokens)
    for mention_idx in range(len(mention_idx_to_chunk_idx_map)):
        assert mention_idx in mention_idx_to_chunk_idx_map
        mention_tokenized_bound = mention_idx_to_chunk_idx_map[mention_idx]
        mention_idxs.append(mention_tokenized_bound)
    for m in range(len(mention_idxs)):
        mention_bounds = mentions[m]
        mention_tok_bounds = mention_idxs[m]
        tokenized_mention = tokenizer.decode(all_token_ids[
            mention_tok_bounds[0]:mention_tok_bounds[1]
        ])
        target_mention = full_example[mention_bounds[0]:mention_bounds[1]].lower()
        try:
            assert tokenized_mention == target_mention
        except:
            # only keep letters and whitespace
            only_letter_tokenized_mention = ""
            only_letter_target_mention = ""
            for char in tokenized_mention:
                if char in string.ascii_letters:
                    only_letter_tokenized_mention += char
            for char in target_mention:
                if char in string.ascii_letters:
                    only_letter_target_mention += char
            print("{} {}".format(tokenized_mention, target_mention))
            try:
                assert only_letter_tokenized_mention == only_letter_target_mention
            except:
                import pdb
                pdb.set_trace()
    return all_token_ids, mention_idxs

In [5]:
def to_elq_tokenized(sample):
#     if tokenizer is None:
#         tokenizer = BertTokenizer.from_pretrained(
#             "bert-base-uncased", do_lower_case=True,
#         )

    res = {}
    res['id'] = sample['doc_id']
    res['text'] = ' '.join(sample['text'])
    
    # generate char indice 'mentions'
    assert len(sample['text'])==len(sample['start_idxs'])==len(sample['end_idxs'])
    char_cnt = 0
    mentions = []
    new_mention = []
    open_mention = False
    for i, word in enumerate(sample['text']):
        if sample['start_idxs'][i]==1:
            new_mention.append(char_cnt)
        char_cnt += len(word) + 1
        if sample['end_idxs'][i]==1:
            new_mention.append(char_cnt-1)
            mentions.append(new_mention)
            new_mention = []
    # sanity check
    assert len(mentions)==len(sample['mentions'])
    for i in range(len(mentions)):
        try:
            assert res['text'][mentions[i][0]:mentions[i][1]]==sample['mentions'][i]
        except:
            print(res['text'][mentions[i][0]:mentions[i][1]], ' ', sample['mentions'][i])
    res['mentions'] = mentions
    
    # generate tokenized texts and mention bounds
    # code from Belinda: create_all_entity_finetuning.py
    all_token_ids, mention_idxs = get_tokenized_bounds(res['mentions'], res['text'], tokenizer)
    res['tokenized_text_ids'] = all_token_ids
    res['tokenized_mention_idxs'] = mention_idxs
    assert len(mention_idxs)==len(sample['wiki_titles'])
    
    res['label_id'] = sample['wiki_ids']
    res['entity'] = sample['wiki_titles']
    res['label'] = sample['wiki_contexts']
    
    return res

In [6]:
in_fpath = 'AIDA-YAGO2-en_desc'
out_fpath = in_fpath+'-ELQ'

fnames = ['train.json', 'dev.json', 'test.json']
num_longs = []

for fname in fnames:
    in_fname = os.path.join(in_fpath, fname)
    with open(in_fname) as fin:
        orig_data = json.load(fin)
    
    elq_tokenized = []
    for sample in tqdm(orig_data):
        elq_sample = to_elq_tokenized(sample)
        elq_tokenized.append(elq_sample)
    
    fname = fname+'l'
    out_fname = os.path.join(out_fpath, fname)
    
    num_long = []
    with open(out_fname, 'w') as wf:
        for i, example in tqdm(enumerate(elq_tokenized)):
            if len(example['tokenized_text_ids']) > 512:
                num_long.append(i)
            b = wf.write(json.dumps(example) + "\n")
    num_longs.append(num_long)

  3%|▎         | 32/946 [00:00<00:06, 141.10it/s]

u. s. u.s.
eliahu ben - elissar eliahu ben-elissar
u. n. u.n.
an - nahar an-nahar
sud - ptt sud-ptt
u. s. u.s.
d. a. weibring d.a. weibring
shi - ting wang shi-ting wang
u. s. u.s.
u. s. u.s.
u. s. u.s.


  5%|▌         | 49/946 [00:00<00:10, 88.82it/s] 

weston - super - mare weston-super-mare
inzamam - ul - haq inzamam-ul-haq
f. a. cup f.a. cup
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
mypa - 47 mypa-47
u. s. open u.s. open
u. s. open u.s. open
u. s u.s
u. s. u.s.
jean - philippe fleurian jean-philippe fleurian
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
carl - uwe steeb carl-uwe steeb
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.


  8%|▊         | 73/946 [00:00<00:08, 99.87it/s]

...

 87%|████████▋ | 200/231 [00:01<00:00, 148.94it/s]

u. s. u.s.
u. s. u.s.
us $ us$
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
simone greiner - petter - memm simone greiner-petter-memm
greiner - petter - memm greiner-petter-memm
u. s u.s
u. s. u.s.
ari - pekka nikkola ari-pekka nikkola
kim yoon - man kim yoon-man
kim yoon - man kim yoon-man
u. s. u.s.
u. s. u.s.
nick farr - jones nick farr-jones


100%|██████████| 231/231 [00:01<00:00, 141.67it/s]
0it [00:00, ?it/s]

f. a. challenge cup f.a. challenge cup
u. s. u.s.
n. ireland n.ireland


231it [00:00, 6609.44it/s]


In [7]:
in_fpath = 'AIDA-YAGO2-wiki_content'
out_fpath = in_fpath+'-ELQ'

fnames = ['train.json', 'dev.json', 'test.json']
num_longs = []

for fname in fnames:
    in_fname = os.path.join(in_fpath, fname)
    with open(in_fname) as fin:
        orig_data = json.load(fin)
    
    elq_tokenized = []
    for sample in tqdm(orig_data):
        elq_sample = to_elq_tokenized(sample)
        elq_tokenized.append(elq_sample)
    
    fname = fname+'l'
    out_fname = os.path.join(out_fpath, fname)
    
    num_long = []
    with open(out_fname, 'w') as wf:
        for i, example in tqdm(enumerate(elq_tokenized)):
            if len(example['tokenized_text_ids']) > 512:
                num_long.append(i)
            b = wf.write(json.dumps(example) + "\n")
    num_longs.append(num_long)

  4%|▎         | 35/946 [00:00<00:05, 156.94it/s]

u. s. u.s.
eliahu ben - elissar eliahu ben-elissar
u. n. u.n.
an - nahar an-nahar
as - safir as-safir
ad - diyar ad-diyar
sud - ptt sud-ptt
u. s. u.s.
d. a. weibring d.a. weibring
shi - ting wang shi-ting wang
u. s. u.s.
u. s. u.s.
u. s. u.s.
weston - super - mare weston-super-mare
inzamam - ul - haq inzamam-ul-haq


  5%|▍         | 44/946 [00:00<00:07, 125.99it/s]

f. a. cup f.a. cup
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
mypa - 47 mypa-47
u. s. open u.s. open
u. s. open u.s. open
u. s u.s
u. s. u.s.
jean - philippe fleurian jean-philippe fleurian
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
carl - uwe steeb carl-uwe steeb
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.


  6%|▋         | 61/946 [00:00<00:10, 83.59it/s] 

...

 83%|████████▎ | 192/231 [00:01<00:00, 128.93it/s]

u. s. u.s.
u. s. u.s.
us $ us$
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
u. s. u.s.
simone greiner - petter - memm simone greiner-petter-memm
greiner - petter - memm greiner-petter-memm
u. s u.s
u. s. u.s.
ari - pekka nikkola ari-pekka nikkola
kim yoon - man kim yoon-man
kim yoon - man kim yoon-man
u. s. u.s.
u. s. u.s.


 95%|█████████▌| 220/231 [00:01<00:00, 106.07it/s]

nick farr - jones nick farr-jones
f. a. challenge cup f.a. challenge cup
u. s. u.s.
n. ireland n.ireland


100%|██████████| 231/231 [00:02<00:00, 111.95it/s]
231it [00:00, 5762.91it/s]
